In [51]:
import gym
import numpy as np
import tensorflow as tf

In [52]:
from gym.envs.registration import register
register(
    id='FrozenLakeNotSlippery-v0',
    entry_point='gym.envs.toy_text:FrozenLakeEnv',
    kwargs={'map_name' : '4x4', 'is_slippery': False},
    max_episode_steps=100,
    reward_threshold=0.78, # optimum = .8196
)

Error: Cannot re-register id: FrozenLakeNotSlippery-v0

In [53]:
env = gym.make('FrozenLakeNotSlippery-v0')
s = env.reset()

rewards = []
actions = []
states = [s]
for _ in range(100):
    env.render()
    a = env.action_space.sample()
    s, r, d, _ = env.step(a)
    rewards.append(r)
    actions.append(a)
    states.append(s)
    if d==True:
        break

[2017-06-19 07:01:49,704] Making new env: FrozenLakeNotSlippery-v0



SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG


折扣累积函数实现

In [65]:
num_episodes = 2000
num_exp = 100

discount = 0.99
Q = np.zeros([env.observation_space.n,env.action_space.n])


for i in range(num_episodes):
    s = env.reset()
    r_all = 0
    for j in range(num_exp):
        # 根据Q-table选择最优动作，为开始中加入随机项，其值随次数增多而减小，模拟探索／利用策略
        a = np.argmax(Q[s,] + np.random.randn(1,env.action_space.n)*(1./(i+1)))
        s1, r, d, _ = env.step(a)
        # 更新Q-table
        Q[s,a] = Q[s,a] + (r + discount*np.max(Q[s1,]-Q[s,a]))
        r_all += r
        s = s1
        if d==True:
            break
    
print Q

[[ 0.          1.01010101  0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          1.01010101  0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          1.01010101  0.        ]
 [ 0.          0.          1.01010101  0.        ]
 [ 0.          1.01010101  0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          1.01010101  0.        ]
 [ 0.          0.          0.          0.        ]]


神经网络实现

https://gist.github.com/awjuliani/4d69edad4d0ed9a5884f3cdcf0ea0874#file-q-net-learning-clean-ipynb

In [ ]:
tf.reset_default_graph()

inputs = tf.placeholder(dtype=tf.float32, shape=[16])
out = tf.layers.dense